In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

['training_set_metadata.csv', 'test_set_sample.csv', 'data_note.pdf', 'training_set.csv', 'test_set_metadata.csv', 'sample_submission.csv', 'test_set.csv']


In [2]:
train_data_df = pd.read_csv('../input/training_set.csv')
train_metadata_df = pd.read_csv('../input/training_set_metadata.csv')

In [3]:
train_data_df['flux_ratio_sq'] = np.power(train_data_df['flux'] / train_data_df['flux_err'], 2.0)
train_data_df['flux_by_flux_ratio_sq'] = train_data_df['flux'] * train_data_df['flux_ratio_sq']

In [4]:
data_features = train_data_df.columns[1:]
metadata_features = train_metadata_df.columns[1:]

In [5]:
groupObjects = train_data_df.groupby('object_id')[data_features]

print("Add constant object features")
features = train_metadata_df.drop(['target'], axis=1)

print("Add mean of mutable object features")
features = pd.merge(features, groupObjects.agg('mean'), how='right', on='object_id', suffixes=['', '_mean'])

print("Add sum of mutable object features")
features = pd.merge(features, groupObjects.agg('sum'), how='right', on='object_id', suffixes=['', '_sum'])

print("Add median of mutable features")
features = pd.merge(features, groupObjects.agg('median'), how='right', on='object_id', suffixes=['', '_median'])

print("Add minimum of mutable features")
features = pd.merge(features, groupObjects.agg('min'), how='right', on='object_id', suffixes=['', '_min'])

print("Add maximum of mutable features")
features = pd.merge(features, groupObjects.agg('max'), how='right', on='object_id', suffixes=['', '_max'])

print("Add range of mutable features")
features = pd.merge(features, groupObjects.agg(lambda x: max(x) - min(x)), how='right', on='object_id', suffixes=['', '_range'])

print("Add standard deviation of mutable features")
features = pd.merge(features, groupObjects.agg('std'), how='right', on='object_id', suffixes=['', '_stddev'])

print("Add skew of mutable features")
features = pd.merge(features, groupObjects.agg('skew'), how='right', on='object_id', suffixes=['', '_skew'])

Add constant object features
Add mean of mutable object features
Add sum of mutable object features
Add median of mutable features
Add minimum of mutable features
Add maximum of mutable features
Add range of mutable features
Add standard deviation of mutable features
Add skew of mutable features


In [6]:
features = features.fillna(features.mean())

In [7]:
features

,object_id,ra,decl,gal_l,gal_b,ddf,hostgal_specz,hostgal_photoz,hostgal_photoz_err,distmod,mwebv,mjd,passband,flux,flux_err,detected,flux_ratio_sq,flux_by_flux_ratio_sq,mjd_sum,passband_sum,flux_sum,flux_err_sum,detected_sum,flux_ratio_sq_sum,flux_by_flux_ratio_sq_sum,mjd_median,passband_median,flux_median,flux_err_median,detected_median,flux_ratio_sq_median,flux_by_flux_ratio_sq_median,mjd_min,passband_min,flux_min,flux_err_min,detected_min,flux_ratio_sq_min,flux_by_flux_ratio_sq_min,mjd_max,passband_max,flux_max,flux_err_max,detected_max,flux_ratio_sq_max,flux_by_flux_ratio_sq_max,mjd_range,passband_range,flux_range,flux_err_range,detected_range,flux_ratio_sq_range,flux_by_flux_ratio_sq_range,mjd_stddev,passband_stddev,flux_stddev,flux_err_stddev,detected_stddev,flux_ratio_sq_stddev,flux_by_flux_ratio_sq_stddev,mjd_skew,passband_skew,flux_skew,flux_err_skew,detected_skew,flux_ratio_sq_skew,flux_by_flux_ratio_sq_skew
0,615,349.046051,-61.943836,320.796530,-51.753706,1,0.0000,0.0000,0.0000,41.263961,0.017,60195.438493,2.457386,-123.096998,4.482743,0.946023,8322.924207,-2.727774e+06,2.118879e+07,865,-43330.143249,1577.925390,333,2.929669e+06,-9.601766e+08,60197.63170,2.0,-89.477524,3.835269,1.0,4104.455781,-49159.288979,59750.4229,0,-1100.440063,2.130510,0,1.868938e-01,-2.963473e+07,60624.2132,5,660.626343,12.845472,1,28486.940606,1.500205e+07,873.7903,5,1761.066406,10.714962,1,28486.753713,4.463678e+07,310.024487,1.720797,394.109851,1.744747,0.226295,8802.368516,7.935039e+06,-0.000196,0.022015,-0.349540,1.623740,-3.964492,0.812722,-1.414322
1,713,53.085938,-27.784405,223.525509,-54.460748,1,1.8181,1.6267,0.2552,45.406300,0.007,60256.769299,2.400000,-1.423351,2.359620,0.171429,16.817336,-8.214535e+01,2.108987e+07,840,-498.172760,825.866990,60,5.886068e+03,-2.875087e+04,60263.56795,2.0,-0.873033,1.998217,0.0,6.390247,-0.246718,59825.2600,0,-14.735178,0.639458,0,7.372764e-04,-2.368542e+03,60674.0798,5,14.770886,9.115748,1,202.167338,7.355214e+02,848.8198,5,29.506064,8.476290,1,202.166601,3.104063e+03,310.247414,1.746056,6.471144,1.509888,0.377423,27.958554,3.076335e+02,0.002387,0.054569,0.014989,1.633246,1.751139,3.439423,-3.454554
2,730,33.574219,-6.579593,170.455585,-61.548219,1,0.2320,0.2262,0.0157,40.256100,0.021,60221.948045,2.336364,2.267434,2.471061,0.069697,12.498338,3.171217e+02,1.987324e+07,771,748.253237,815.450286,23,4.124452e+03,1.046502e+05,60223.26235,2.0,0.409172,1.990851,0.0,0.709582,0.023877,59798.3205,0,-19.159811,0.695106,0,2.966172e-04,-7.261569e+01,60652.1660,5,47.310059,11.281384,1,403.400590,1.306587e+04,853.8455,5,66.469870,10.586278,1,403.400293,1.313849e+04,302.781791,1.758750,8.022239,1.721134,0.255022,51.909146,1.577430e+03,0.093703,0.095295,3.177854,1.823726,3.395207,5.480405,5.989138
3,745,0.189873,-45.586655,328.254458,-68.969298,1,0.3037,0.2813,1.1523,40.795100,0.007,60199.831697,2.378917,8.909206,2.555576,0.173789,268.266811,4.100072e+04,2.113014e+07,835,3127.131254,897.007002,61,9.416165e+04,1.439125e+07,60200.07370,2.0,1.035895,1.819875,0.0,1.120752,0.390292,59770.3662,0,-15.494463,0.567170,0,5.540601e-05,-4.269851e+01,60624.0722,5,220.795212,55.892746,1,21895.121071,4.834338e+06,853.7060,5,236.289675,55.325576,1,21895.121015,4.834381e+06,306.144145,1.747328,27.558208,3.537324,0.379469,1754.236011,3.490637e+05,0.032516,0.066341,4.979826,10.741655,1.729154,9.611274,11.141069
4,1124,352.711273,-63.823658,316.922299,-51.059403,1,0.1934,0.2415,0.0176,40.416600,0.024,60195.438493,2.457386,7.145702,2.753004,0.173295,97.511874,8.567041e+03,2.118879e+07,865,2515.287161,969.057329,61,3.432418e+04,3.015599e+06,60197.63170,2.0,1.141288,2.214854,0.0,0.705444,0.285458,59750.4229,0,-16.543753,0.695277,0,1.747338e-06,-4.613587e+01,60624.2132,5,143.600189,11.383690,1,5647.431717,5.324559e+05,873.7903,5,160.143942,10.688413,1,5647.431715,5.325020e+05,310.024487,1.720797,20.051722,1.933837,0.379041,515.365785,5.733142e+04,-0.000196,0.022015,4.406298,1.794938,1.733697,7.868462,7.908174
5,1227,35.683594,-5.379379,

In [8]:
import tensorflow as tf

In [9]:
import keras
from keras.utils import to_categorical

Using TensorFlow backend.


In [10]:
train_metadata_df['target'] = train_metadata_df.target.map({6:0, 15:1, 16:2, 42:3, 52:4, 53:5, 62:6, 64:7, 65:8, 67:9, 88:10, 90:11, 92:12, 95:13})
targets = train_metadata_df['target']

**Engineer new features using Genetic Programming with the gplearn library.**

In [11]:
import gplearn
from gplearn.genetic import SymbolicTransformer

In [12]:
function_set = ['add', 'sub', 'mul', 'div',
                'sqrt', 'log', 'abs', 'neg', 'inv',
                'max', 'min']

gp = SymbolicTransformer(generations=100, population_size=2000,
                         hall_of_fame=100, n_components=10,
                         function_set=function_set,
                         parsimony_coefficient=0.0005,
                         max_samples=0.9, verbose=1,
                         random_state=0, n_jobs=3)

gp.fit(features.drop('object_id', axis=1).values, targets.values)

    |    Population Average   |             Best Individual              |
---- ------------------------- ------------------------------------------ ----------
 Gen   Length          Fitness   Length          Fitness      OOB Fitness  Time Left
   0     9.96 0.06101262578887118        2 0.40355472498294814 0.37651519147203605     10.37m
   1     6.98 0.171320654249389        7 0.4083573064501234 0.41626336471212527      8.57m
   2     5.92 0.2870241429492127       10 0.4275676419367279 0.4182045821485891      8.10m
   3     3.03 0.37542760695217836       10 0.43116215442625727 0.3864155027134665      7.54m
   4     4.19 0.3469917880508957       10 0.4330574065379512 0.3687495938867573      7.26m
   5     7.48 0.299576563912424       16 0.4432884377660313 0.43818103385216645      7.19m
   6     9.27 0.31788461503594406       16 0.4486610086674179 0.3429771661448803      7.20m
   7     9.75 0.34324061391948973       28 0.45298365299342164 0.4340214504600353      7.19m
   8     12.3 0.358

SymbolicTransformer(const_range=(-1.0, 1.0),
          function_set=['add', 'sub', 'mul', 'div', 'sqrt', 'log', 'abs', 'neg', 'inv', 'max', 'min'],
          generations=100, hall_of_fame=100, init_depth=(2, 6),
          init_method='half and half', max_samples=0.9, metric='pearson',
          n_components=10, n_jobs=3, p_crossover=0.9,
          p_hoist_mutation=0.01, p_point_mutation=0.01,
          p_point_replace=0.05, p_subtree_mutation=0.01,
          parsimony_coefficient=0.0005, population_size=2000,
          random_state=0, stopping_criteria=1.0, tournament_size=20,
          verbose=1, warm_start=False)

In [13]:
engineered_features = gp._programs

for i in range(len(engineered_features)):
    for engineered_feature in engineered_features[i]:
        if engineered_feature != None:
            print(engineered_feature)

sqrt(X8)
max(X17, X5)
div(X53, X13)
inv(X27)
sqrt(X60)
sqrt(X5)
abs(min(neg(X5), mul(X52, X20)))
min(X10, X60)
add(X5, max(sub(X4, min(X34, X8)), neg(X47)))
neg(log(sqrt(X37)))
min(X18, X8)
sub(sqrt(neg(min(X56, X5))), inv(max(sub(X43, X46), div(X8, X34))))
neg(sqrt(div(min(X41, X5), log(X64))))
log(sub(X29, X37))
min(div(X25, X45), sub(X34, X25))
sub(X59, X8)
min(X61, X52)
neg(X60)
log(add(log(max(X1, X13)), sqrt(div(X36, X49))))
mul(X5, X10)
sub(mul(add(neg(neg(mul(X13, X35))), neg(inv(add(X48, X19)))), inv(max(inv(inv(X31)), sqrt(sub(X56, X18))))), sqrt(max(log(inv(sub(X43, X64))), abs(add(neg(X65), abs(X65))))))
abs(X18)
sub(X60, X53)
sqrt(X5)
log(log(max(log(X46), abs(X25))))
neg(sub(X5, sqrt(X32)))
min(X2, X65)
max(X6, neg(X50))
sub(X5, X22)
min(X65, X25)
inv(X34)
min(0.845, X65)
inv(X11)
abs(X53)
sub(min(log(log(log(abs(X58)))), inv(abs(log(max(X22, X9))))), log(add(mul(div(min(X59, X45), sqrt(X32)), mul(min(X27, X5), mul(X59, X42))), div(abs(inv(X5)), mul(div(X13, 0.380), add(X

In [14]:
new_features = pd.DataFrame(gp.transform(features.drop('object_id', axis=1).values))
features = pd.concat([features, new_features], axis=1, join_axes=[features.index])

In [15]:
features

,object_id,ra,decl,gal_l,gal_b,ddf,hostgal_specz,hostgal_photoz,hostgal_photoz_err,distmod,mwebv,mjd,passband,flux,flux_err,detected,flux_ratio_sq,flux_by_flux_ratio_sq,mjd_sum,passband_sum,flux_sum,flux_err_sum,detected_sum,flux_ratio_sq_sum,flux_by_flux_ratio_sq_sum,mjd_median,passband_median,flux_median,flux_err_median,detected_median,flux_ratio_sq_median,flux_by_flux_ratio_sq_median,mjd_min,passband_min,flux_min,flux_err_min,detected_min,flux_ratio_sq_min,flux_by_flux_ratio_sq_min,mjd_max,passband_max,flux_max,flux_err_max,detected_max,flux_ratio_sq_max,flux_by_flux_ratio_sq_max,mjd_range,passband_range,flux_range,flux_err_range,detected_range,flux_ratio_sq_range,flux_by_flux_ratio_sq_range,mjd_stddev,passband_stddev,flux_stddev,flux_err_stddev,detected_stddev,flux_ratio_sq_stddev,flux_by_flux_ratio_sq_stddev,mjd_skew,passband_skew,flux_skew,flux_err_skew,detected_skew,flux_ratio_sq_skew,flux_by_flux_ratio_sq_skew,0,1,2,3,4,5,6,7,8,9
0,615,349.046051,-61.943836,320.796530,-51.753706,1,0.0000,0.0000,0.0000,41.263961,0.017,60195.438493,2.457386,-123.096998,4.482743,0.946023,8322.924207,-2.727774e+06,2.118879e+07,865,-43330.143249,1577.925390,333,2.929669e+06,-9.601766e+08,60197.63170,2.0,-89.477524,3.835269,1.0,4104.455781,-49159.288979,59750.4229,0,-1100.440063,2.130510,0,1.868938e-01,-2.963473e+07,60624.2132,5,660.626343,12.845472,1,28486.940606,1.500205e+07,873.7903,5,1761.066406,10.714962,1,28486.753713,4.463678e+07,310.024487,1.720797,394.109851,1.744747,0.226295,8802.368516,7.935039e+06,-0.000196,0.022015,-0.349540,1.623740,-3.964492,0.812722,-1.414322,2.777064,2.683820,17.504267,2.777064,3.089794,2.777064,17.504267,17.504267,2.777064,3.125388
1,713,53.085938,-27.784405,223.525509,-54.460748,1,1.8181,1.6267,0.2552,45.406300,0.007,60256.769299,2.400000,-1.423351,2.359620,0.171429,16.817336,-8.214535e+01,2.108987e+07,840,-498.172760,825.866990,60,5.886068e+03,-2.875087e+04,60263.56795,2.0,-0.873033,1.998217,0.0,6.390247,-0.246718,59825.2600,0,-14.735178,0.639458,0,7.372764e-04,-2.368542e+03,60674.0798,5,14.770886,9.115748,1,202.167338,7.355214e+02,848.8198,5,29.506064,8.476290,1,202.166601,3.104063e+03,310.247414,1.746056,6.471144,1.509888,0.377423,27.958554,3.076335e+02,0.002387,0.054569,0.014989,1.633246,1.751139,3.439423,-3.454554,10.798996,11.017052,25.115983,11.012528,10.799000,10.798996,25.332840,25.333754,11.010558,11.011899
2,730,33.574219,-6.579593,170.455585,-61.548219,1,0.2320,0.2262,0.0157,40.256100,0.021,60221.948045,2.336364,2.267434,2.471061,0.069697,12.498338,3.171217e+02,1.987324e+07,771,748.253237,815.450286,23,4.124452e+03,1.046502e+05,60223.26235,2.0,0.409172,1.990851,0.0,0.709582,0.023877,59798.3205,0,-19.159811,0.695106,0,2.966172e-04,-7.261569e+01,60652.1660,5,47.310059,11.281384,1,403.400590,1.306587e+04,853.8455,5,66.469870,10.586278,1,403.400293,1.313849e+04,302.781791,1.758750,8.022239,1.721134,0.255022,51.909146,1.577430e+03,0.093703,0.095295,3.177854,1.823726,3.395207,5.480405,5.989138,7.057667,6.541526,22.416334,6.527450,7.057730,6.896398,21.956242,21.901629,6.597574,6.542972
3,745,0.189873,-45.586655,328.254458,-68.969298,1,0.3037,0.2813,1.1523,40.795100,0.007,60199.831697,2.378917,8.909206,2.555576,0.173789,268.266811,4.100072e+04,2.113014e+07,835,3127.131254,897.007002,61,9.416165e+04,1.439125e+07,60200.07370,2.0,1.035895,1.819875,0.0,1.120752,0.390292,59770.3662,0,-15.494463,0.567170,0,5.540601e-05,-4.269851e+01,60624.0722,5,220.795212,55.892746,1,21895.121071,4.834338e+06,853.7060,5,236.289675,55.325576,1,21895.121015,4.834381e+06,306.144145,1.747328,27.558208,3.537324,0.379469,1754.236011,3.490637e+05,0.032516,0.066341,4.979826,10.741655,1.729154,9.611274,11.141069,8.338201,7.418296,23.598172,7.399173,8.338367,8.098787,22.747631,22.680388,7.487661,7.420442
4,1124,352.711273,-63.823658,316.922299,-51.059403,1,0.1934,0.2415,0.0176,40.416600,0.024,60195.438493,2.457386,7.145702,2.753004,0.173295,97.511874,8.567041e+03,2.118879e+07,865,2515.287161,969.057329,61,3.432418e+04,3.015599e+06,60

In [16]:
targets = to_categorical(targets)

In [17]:
features = features.drop(['object_id'], axis=1).values

In [18]:
import sklearn
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler(feature_range=(-1, 1)).fit(features)
features = scaler.transform(features)

In [19]:
train_features = features [:np.int32(0.8*len(features))]
train_targets = targets [:np.int32(0.8*len(features))]

val_features = features[np.int32(0.8*len(features)):]
val_targets = targets[np.int32(0.8*len(features)):]

**Build a simple Deep Neural Network using Dense, Dropout and BatchNormalization layers in Keras**

In [20]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, BatchNormalization
from keras.regularizers import L1L2

In [21]:
model = Sequential()

model.add(Dense(30, activation='relu')) # kernel_regularizer=L1L2(l1=0.00, l2=0.01), bias_regularizer=L1L2(l1=0.00, l2=0.01)
model.add(BatchNormalization())
model.add(Dropout(0.2))


model.add(Dense(40, activation='relu')) # kernel_regularizer=L1L2(l1=0.00, l2=0.01), bias_regularizer=L1L2(l1=0.00, l2=0.01)
model.add(BatchNormalization())
model.add(Dropout(0.2))

model.add(Dense(50, activation='relu')) # kernel_regularizer=L1L2(l1=0.00, l2=0.01), bias_regularizer=L1L2(l1=0.00, l2=0.01)
model.add(BatchNormalization())
model.add(Dropout(0.2))

model.add(Dense(10, activation='relu')) # kernel_regularizer=L1L2(l1=0.00, l2=0.01), bias_regularizer=L1L2(l1=0.00, l2=0.01)
model.add(BatchNormalization())
model.add(Dropout(0.2))

model.add(Dense(targets.shape[1], activation='softmax')) # kernel_regularizer=L1L2(l1=0.00, l2=0.01), bias_regularizer=L1L2(l1=0.00, l2=0.01)

**Compile the model with Categorical Cross Entropy and RMSProp as the loss function and optimizer of the model respectively. **

In [22]:
model.compile(loss='categorical_crossentropy', optimizer='rmsprop')

Instructions for updating:
Colocations handled automatically by placer.


**Train the model and check performance on unseen data with a validation data set.**

In [23]:
model.fit(train_features, train_targets, validation_data=(val_features, val_targets), epochs=500)

Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
Use tf.cast instead.
Train on 6278 samples, validate on 1570 samples
Epoch 1/500
6278/6278 [==============================] - 2s 294us/step - loss: 2.4514 - val_loss: 1.9995
Epoch 2/500
6278/6278 [==============================] - 1s 97us/step - loss: 1.8569 - val_loss: 1.6588
Epoch 3/500
6278/6278 [==============================] - 1s 98us/step - loss: 1.5777 - val_loss: 1.4568
Epoch 4/500
6278/6278 [==============================] - 1s 93us/step - loss: 1.4392 - val_loss: 1.3510
Epoch 5/500
6278/6278 [==============================] - 1s 93us/step - loss: 1.3344 - val_loss: 1.4190
Epoch 6/500
6278/6278 [==============================] - 1s 92us/step - loss: 1.2841 - val_loss: 1.2617
Epoch 7/500
6278/6278 [==============================] - 1s 91us/step - loss: 1.2412 - val_loss: 1.2112
Epoch 8/500
6278/6278 [==============================] - 1s 

**Iterate over the test data in batches and make predictions on each batch using the trained model. **

In [24]:
import time

test_metadata_df = pd.read_csv('../input/test_set_metadata.csv')

# print("Add constant object features")
test_metadata_df = test_metadata_df.fillna(test_metadata_df.mean())

predictions = []
object_ids = []

chunks = 5000000
total = 0

for i_c, test_data_df in enumerate(pd.read_csv('../input/test_set.csv', chunksize=chunks, iterator=True)):
    startTime = time.time()
    
    test_data_df['flux_ratio_sq'] = np.power(test_data_df['flux'] / test_data_df['flux_err'], 2.0)
    test_data_df['flux_by_flux_ratio_sq'] = test_data_df['flux'] * test_data_df['flux_ratio_sq']
    
    groupObjects = test_data_df.fillna(test_data_df.mean()).groupby('object_id')[data_features]

    # print("Add mean of mutable object features")
    features = groupObjects.agg('mean')
    
    # print("Add sum of mutable object features")
    features = pd.merge(features, groupObjects.agg('sum'), how='right', on='object_id', suffixes=['', '_sum'])

    # print("Add median of mutable features")
    features = pd.merge(features, groupObjects.agg('median'), how='right', on='object_id', suffixes=['', '_median'])

    # print("Add minimum of mutable features")
    features = pd.merge(features, groupObjects.agg('min'), how='right', on='object_id', suffixes=['', '_min'])

    # print("Add maximum of mutable features")
    features = pd.merge(features, groupObjects.agg('max'), how='right', on='object_id', suffixes=['', '_max'])

    # print("Add range of mutable features")
    features = pd.merge(features, groupObjects.agg(lambda x: max(x) - min(x)), how='right', on='object_id', suffixes=['', '_range'])

    # print("Add standard deviation of mutable features")
    features = pd.merge(features, groupObjects.agg('std'), how='right', on='object_id', suffixes=['', '_stddev'])
    
    # print("Add skew of mutable features")
    features = pd.merge(features, groupObjects.agg('skew'), how='right', on='object_id', suffixes=['', '_skew'])
    
    test_features = pd.merge(test_metadata_df, features, on='object_id')
    
    new_features = pd.DataFrame(gp.transform(test_features.drop('object_id', axis=1).values))
    test_features = pd.concat([test_features, new_features], axis=1, join_axes=[test_features.index])
    
    object_ids.extend(list(test_features['object_id']))
    test_features = test_features.drop(['object_id'], axis=1).values
    
    test_features = scaler.transform(test_features)
    total = total + len(test_features)
    
    predictions.extend(model.predict(test_features))

    endTime = time.time()
    
    print("Iteration : " + str(i_c))
    print("Time taken : " + str(endTime - startTime) + " s")
    print("Total objects predicted on : " + str(total))
    print("")

Iteration : 0
Time taken : 20.060746669769287 s
Total objects predicted on : 15138

Iteration : 1
Time taken : 19.694578886032104 s
Total objects predicted on : 30322

Iteration : 2
Time taken : 31.9548499584198 s
Total objects predicted on : 65287

Iteration : 3
Time taken : 34.291847944259644 s
Total objects predicted on : 104345

Iteration : 4
Time taken : 35.354835987091064 s
Total objects predicted on : 143394

Iteration : 5
Time taken : 34.904067516326904 s
Total objects predicted on : 182473

Iteration : 6
Time taken : 35.75892472267151 s
Total objects predicted on : 221529

Iteration : 7
Time taken : 34.752387285232544 s
Total objects predicted on : 260622

Iteration : 8
Time taken : 34.43295955657959 s
Total objects predicted on : 299702

Iteration : 9
Time taken : 34.78434610366821 s
Total objects predicted on : 338757

Iteration : 10
Time taken : 34.65516996383667 s
Total objects predicted on : 377854

Iteration : 11
Time taken : 34.2791314125061 s
Total objects predicted on

ValueError: Input contains NaN, infinity or a value too large for dtype('float64').

**Prepare predictions for submission.**

In [25]:
predictions = pd.DataFrame(predictions)

In [26]:
predictions.columns = ['class_6', 'class_15', 'class_16', 'class_42', 'class_52', 'class_53', 'class_62', 'class_64', 'class_65', 'class_67', 'class_88', 'class_90', 'class_92', 'class_95']

**The class-99 probability is calculated as 1 minus the maximum probability predicted by the model.**

In [27]:
predictions['class_99'] = 1 - predictions.max(axis=1)
predictions['object_id'] = object_ids

In [28]:
predictions

,class_6,class_15,class_16,class_42,class_52,class_53,class_62,class_64,class_65,class_67,class_88,class_90,class_92,class_95,class_99,object_id
0,0.000051,0.028701,0.000108,0.717051,0.041427,0.000013,0.124199,0.000363,0.000128,0.005569,0.002975,0.079129,0.000002,0.000285,0.282949,13
1,0.000279,0.030820,0.000307,0.180384,0.051605,0.000110,0.084676,0.002681,0.001193,0.069933,0.000504,0.576922,0.000301,0.000286,0.423078,14
2,0.000312,0.027706,0.000517,0.146094,0.049208,0.000096,0.065483,0.001571,0.000945,0.062963,0.000560,0.643872,0.000389,0.000284,0.356128,17
3,0.000278,0.027347,0.000486,0.209226,0.074196,0.000092,0.117068,0.002407,0.001338,0.089087,0.000484,0.477286,0.000550,0.000155,0.522714,23
4,0.000476,0.032686,0.000066,0.092843,0.013037,0.000145,0.022669,0.000228,0.000309,0.013941,0.000777,0.820793,0.000030,0.001999,0.179207,34
5,0.000432,0.030789,0.000686,0.159903,0.054629,0.000128,0.078079,0.001835,0.001465,0.071741,0.000527,0.598962,0.000548,0.000275,0.401038,35
6,0.000220,0.021163,0.000417,0.147283,0.058705,0.000061,0.072086,0.001259,0.000794,0.071727,0.000439,0.625307,0.000391,0.000147,0.374693,43
7,0.000231,0.031869,0.000252,0.190727,0.045217,0.000125,0.079099,0.003792,0.001226,0.063354,0.000524,0.583038,0.000189,0.000356,0.416962,50
8,0.000323,0.030336,0.001171,0.261351,0.079141,0.000089,0.113053,0.001576,0.001279,0.062926,0.000912,0.447241,0.000395,0.000207,0.552759,60
9,0.000354,0.034280,0.001000,0.291386,0.083873,0.000125,0.147591,0.002252,0.001568,0.073374,0.000826,0.362651,0.000518,0.000203,0.637349,69


In [29]:
predictions.to_csv('plasticc_submission_file.csv', index=False)